# PageRank

<br>
<br>
<center>
<img src="images/Page-et-Brin.jpeg" alt="Larry Page and Sergey Brin" width="450">
<br>
Larry Page and Sergey Brin


<div style="visibility: hidden"> 
Source:
http://pixshark.com/sergey-brin-and-larry-page.htm
</div>

In [3]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
# import libraries
import numpy as np
import matplotlib as mp
import pandas as pd
import matplotlib.pyplot as plt
import laUtilities as ut
import slideUtilities as sl
import demoUtilities as dm
import pandas as pd
from datetime import datetime
from IPython.display import Image
from IPython.display import display_html
from IPython.display import display
from IPython.display import Math
from IPython.display import Latex
from IPython.display import HTML
reload(dm)
reload(ut)
reload(sl)
print ''

In [4]:
%%html
<style>
 .container.slides .celltoolbar, .container.slides .hide-in-slideshow {
    display: None ! important;
}
</style>

%Set up useful MathJax (Latex) macros.
%See http://docs.mathjax.org/en/latest/tex.html#defining-tex-macros
%These are for use in the slideshow
$\newcommand{\mat}[1]{\left[\begin{array}#1\end{array}\right]}$
$\newcommand{\vx}{{\mathbf x}}$
$\newcommand{\R}{{\mathbb{R}}}$
$\newcommand{\vu}{{\mathbf u}}$
$\newcommand{\vv}{{\mathbf v}}$
$\newcommand{\col}{{\operatorname{Col}}}$
$\newcommand{\nul}{{\operatorname{Nul}}}$
$\newcommand{\vb}{{\mathbf b}}$
$\newcommand{\va}{{\mathbf a}}$
$\newcommand{\ve}{{\mathbf e}}$
$\newcommand{\setb}{{\mathcal{B}}}$
$\newcommand{\rank}{{\operatorname{rank}}}$
$\newcommand{\vp}{{\mathbf p}}$

Today we'll study an algorithm that is probably important in your life: Google's PageRank.

By way of history: the World Wide Web starting becoming widely used in 1994.

By 1998 the Web had become an indispensable information resource. However the problem of effectively searching the Web for relevant information was not well addressed.  A number of large search engines were available, with names that are now forgotten: Alta Vista, Lycos, Excite, and others. At present, most of them are no longer in existence, because Google emerged in 1998 and came to dominate Web search almost overnight.

How did this happen?

As background: a typical search engine uses a two-step process to retrieve pages related to a user’s query. In
the first step, basic text processing is done to find all documents that contain the query terms. 
Due to the massive size of the Web, this first step can result in many thousands of retrieved pages related to the query.   

The problem that Google solves better than the search engines of the mid 1990’s concerns the __ordering__ in which the resulting search results are presented.  This is the crucial factor in utility.  A user wants to find the "correct" or "best" item at the top of the search results.

By displaying the most relevant pages at the top of the list returned each query, Google makes its search results very useful. The algorithm that gave Google this advantage is called PageRank.

Around 1998, the limitations of standard search engines, which just used term frequency, we becoming apparent.  A number of researchers were thinking about using additional sources of information to "rate" pages.

The key idea that a number of researchers hit on was this: _links are endorsements._  When a first page contains a link to a second page, that is an indication that the author of the first page thinks the second page is worth looking at.  If the first and second pages both contain the same query terms, it is likely that the second page is an important page with respect to that query term.

Consider a set of web pages, for a single query term (say "car manufacturers") with this linking structure:

<img src="images/hub-authority.png" alt="Hubs" width="250">

It may be clear that the links between pages contain and encode information.  But what is the best way to extract that information in the form of rankings?

What was the insight that Brin and Page used?

From “The PageRank citation ranking: Bringing order to the Web” (1998):
> PageRank can be thought of as a model of user behavior. We assume there is a “random surfer” who is given a web page at random and keeps clicking on links, never hitting “back” but eventually gets bored and starts on another random page. The probability that the random surfer visits a page is its PageRank.

What does this have to do with Markov Chains and eigenvectors?

## Random Walks

We start with the notion of a __random walk.__

A random walk is a model of many sorts of processes that occur on graphs.

Let us fix a graph $G$.  A random walk (sometimes called a "drunkard's walk") models the movement of an object on this graph.

We assume that the object moves from node to node in $G$, one move per time step $t.$  At time $t$ the object is at node $k$ (say) and at the next time $t+1$ it moves to another node chosen __at random__ from among the outgoing edges.

To fix ideas for our initial discussion, we will assume that $G$ is the line graph:

<img src="images/Lay-fig-10-3.jpg" alt="Line Graph" width="450">

This is a graph in which each node is connected to two neighbors.  It's natural to identify the nodes with the integers $k = 1,\dots,n.$

What happens at the endpoints of the graph (nodes 1 and $n$) must be specified.

One possibility is for the object to remain fixed at that location.   This is called a __random walk with absorbing boundaries.__

Another possibility is for the object to bounce back one unit when an endpoint is reached.   This is called a __random walk with reflecting boundaries.__

We can also set a particular probability $1-p$ of moving "to the right" (from $k$ to $k+1$) and $p$ of moving "to the left" (from $k$ to $k-1$).

We can capture this process as a Markov Chain.  

__Example.__  A random walk on $\{1,2,3,4,5\}$ with absorbing boundaries has a transition matrix of 

$$P=\mat{{ccccc}1&p&0&0&0\\0&0&p&0&0\\0&1-p&0&p&0\\0&0&1-p&0&0\\0&0&0&1-p&1}$$

__Example.__ Gambler's Ruin.   Consider a very simple casino game.  A gambler (with some money to lose) flips a coin and calls heads or tails.  If the gambler is correct, he wins a dollar.  If he is wrong, he loses a dollar.  Suppose that the gambler will quit the game when she has either won $n$ dollars or lost all of her money.

Suppose that $n=7$ and the gambler starts with \$4.  The gambler's winnings must move up or down one dollar with each coin flip, and once the gambler's winnings reach 0 or 7, they do not change any more since the gambler has quit the game.  

Such a process may be modeled as a random walk on $\{0,1,2,3,4,5,6,7\}$ with absorbing boundaries.   Since a move up or down is equally likely in this case, $p = 1/2$.

This transition matrix is not regular, and there is not a single steady-state to which the chain surely converges.   However there are two steady-states, each corresponding to absorption at one boundary, the chain will surely converge to one or the other.

For example, if $p=0.45$, we find that the probability that the gambler will lose all her money to be $0.4$.

In [36]:
sl.hide_code_in_slideshow()
p = 0.45
A = np.array([[1,p,0,0,0],[0,0,p,0,0],[0,1-p,0,p,0],[0,0,1-p,0,0],[0,0,0,1-p,1]])
B = A.copy()
for i in range(100):
    B = A.dot(B)
# print B.dot(np.array([0,0,1,0,0]))

Now let's consider a random walk on a more interesting graph:

<img src="images/Lay-fig-10-4.jpg" alt="Another Graph" height="200">

Again, at each node there is an equal probability of departing to any adjacent node.  This is called a __simple random walk on a graph.__

The transition matrix associated with this graph is

$$A = \mat{{ccccccc}
0&1/3&1/4&0&0&0&0\\
1/2&0&1/4&0&1/2&0&0\\
1/2&1/3&0&1&0&1/3&0\\
0&0&1/4&0&0&0&0\\
0&1/3&0&0&0&1/3&0\\
0&0&1/4&0&1/2&0&1\\
0&0&0&0&0&1/3&0}$$

This graph is regular, so the transition matrix converges to a single steady state.  (It has only one eigenvalue of 1.)

Its principal eigenvector is proportional to: $\mat{{c}2\\3\\4\\1\\2\\3\\1}.$  This represents the steady-state distribution.

Notice anything?  (Look at $G$).

In [35]:
sl.hide_code_in_slideshow()
A = np.array([
[0,1./3,1./4,0,0,0,0],
[1./2,0,1./4,0,1./2,0,0],
[1./2,1./3,0,1,0,1./3,0],
[0,0,1./4,0,0,0,0],
[0,1./3,0,0,0,1./3,0],
[0,0,1./4,0,1./2,0,1],
[0,0,0,0,0,1./3,0]])
w,v = np.linalg.eig(A)
# print v[:,0]*(20./3)

Another interesting object on which to walk randomly is a __directed__ graph.  In this graph, all edges are "one-way streets" -- nodes are joined not by lines but by arrows.   The chain can move from vertex to vertex, but only in the directions allowed by the arrows.

An example of a directed graph is


<img src="images/deeper-pagerank-fig.jpg" alt="Directed Graph" height="200">

The transition matrix for this graph is:

$$A = \mat{{cccccc}0&1/2&1/2&0&0&0\\0&0&0&0&0&0\\
1/3&1/3&0&0&1/3&0\\
0&0&0&0&1/2&1/2\\
0&0&0&1&0&0}$$

## PageRank

There are many ways to make use of the link structure to infer which pages are most important to return at the top of the search results.

The simplest is to consider a page is "important" if many "important" pages link to it.

This can be captured in terms of a random walk.

Now we are ready to understand what Page and Brin were saying in 1998:

> PageRank can be thought of as a model of user behavior. We assume there is a “random surfer” who is given a web page at random and keeps clicking on links, never hitting “back” but eventually gets bored and starts on another random page. The probability that the random surfer visits a page is its PageRank.

Intuitively, the a random surfer should spend more time at "important" pages and less time at unimportant pages.

The way to interpret this precisely is:

We construct a Markov Chain from the graph that encodes the connections between Web pages that are retrieved for a particular query.   Then we rank-order the results according to their probability in the steady state.

So let's try to make this work and see what happens.

## Computing PageRank: the Power Method